<a href="https://colab.research.google.com/github/GageBrady92/CS361/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the TensorFlow git repo to make use of the API.

In [ ]:
!git clone https://github.com/tensorflow/models.git

Install tensorflow API

In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .


Quick test to ensure API installed properly

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

Get files from kaggle data set and make new folders. These files have been annotated into .xml files and then specific fields from .xml converted to single CSV file for training and testing.

In [ ]:
! pip install -q kaggle

This command allows the connection to kaggle by creating an API token of the kaggle account and putting into a JSON. Only need it when the file is not present.

In [ ]:
from google.colab import files

files.upload()

Use this to make directory, go into directory and unzip files to be used. Have the image files, annotations, CSV and python code to convert data to binary for tensorflow. This saves time by having all that is needed in one spot to pull from.

In [ ]:
%%bash
mkdir /content/dataset
cd /content/dataset
kaggle datasets download -d gagebrady/brain-mri-tumor-data --unzip

In [ ]:
%cd /content/

These commands convert CSV of image annotations to be passed to tensorflow object detection API.

In [ ]:
!python dataset/generate_tf_records.py -l /content/labelmap.pbtxt -o dataset/train.record -i dataset/images -csv dataset/train_labels.csv
!python dataset/generate_tf_records.py -l /content/labelmap.pbtxt -o dataset/test.record -i dataset/images -csv dataset/test_labels.csv

Download and set up the model that will be trained.

In [ ]:
%cd /content
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz
!rm mobilenet_v2.tar.gz

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

Creating paths and parameters and then configuring training file.
Only checking for one class, which is tumors. Keeping batch size small to avoid memory issues that will stop the execution.
This also configures for the specific model being used, mobilenet_v2.

In [ ]:
num_classes = 1
batch_size = 2
num_steps = 7500
num_eval_steps = 1000

train_record_path = '/content/dataset/train.record'
test_record_path = '/content/dataset/test.record'
model_dir = '/content/training/'
labelmap_path = '/content/labelmap.pbtxt'

pipeline_config_path = 'mobilenet_v2.config'
fine_tune_checkpoint = '/content/mobilenet_v2/mobilenet_v2.ckpt-1'



In [ ]:
import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:

  # Set labelmap path
  config = re.sub('label_map_path: ".*?"',
             'label_map_path: "{}"'.format(labelmap_path), config)

  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
                  'input_path: "{}"'.format(train_record_path), config)

  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
                  'input_path: "{}"'.format(test_record_path), config)

  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)

  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)

  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)

  f.write(config)

This code starts the process of training the model. With current setup, doing batches of 2 with 7500 steps.

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

This checks the models performance and calculates loss. Lower the loss, better the accuracy

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}

This will create simple graphs to represent the model training.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/'